In [1]:
import data
import ast_util
import ast
import evaluation

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/omkar/.config/sagemaker/config.yaml


In [3]:
datasets = {
    "list": "list_master_finetune_full",
    "tree": "tree_master_finetune_full",
    "lseg": "lseg_master_finetune_full"
}

training_examples = {
    "list": [
        "prepend",
        "append",
        "remove_first",
        "join_lists",
        "contains",
        "remove",
        "index_of",
        "drop",
        "reverse",
        "insert_sorted",
        "insertion_sort",
        "merge",
    ],
    "tree": [
        "val_head",
        "count",
        "sum",
        "contains",
        "inorder",
        "min",
        "mirror",
        "subtree",
    ],
    "lseg": [
        "join",
        "prepend",
        "remove_first",
        "contains",
        "contains_iter",
        "insert",
        "insert_iter",
        "append",
        "append_iter",
        "insert_sorted",
        "insertion_sort",
        "insertion_sort_iter",
        "merge",
        "split_iter",
    ],
}

for KEY in datasets:
    dataset = data.Data("config.json", f"nagini_examples/{datasets[KEY]}")
    ev = evaluation.Evaluation(dataset)
    # examples = dataset.get_list_of_examples(KEY)
    examples = training_examples[KEY]
    print(examples)
    for example in examples:
        master_program = dataset.get_master_example(KEY, example)
        if master_program is None:
            verified_program = dataset.get_example(KEY, example, "verified")
            master_program = verified_program
        master_ast = ast.parse(master_program, type_comments=True)
        print(ast.unparse(master_ast))
        program_powerset = ast_util.dataset_generator(master_ast)
        for i, program_ast in enumerate(program_powerset):
            program = ast.unparse(program_ast)
            verif_error = str(ev.verify_program_snippet(KEY, program, method_name=example))
            dataset.add_example_verified_ptr(f"{example}_{i}", program, verif_error, f"{example}_verified.txt", KEY)

['contains_iter']
def contains_iter(first: Node, last: Optional[Node], val: int) -> bool:
    """Check if the list contains a node with value val."""
    Requires(lseg(first, last))
    Ensures(lseg(first, last))
    ptr = first # type: Optional[Node]
    result = False
    Fold(lseg(first, ptr))
    while ptr is not None and ptr is not last:
        Invariant(lseg(first, ptr))
        Invariant(lseg(ptr, last))
        Unfold(lseg(ptr, last))
        if Unfolding(lseg(ptr, last), ptr.val) == val:
            Fold(lseg(ptr, last))
            result = True
            break
        Unfold(lseg(ptr, last))
        tmp = ptr
        ptr = ptr.next
        Fold(lseg(ptr, ptr))
        Fold(lseg(tmp, ptr))
        join(first, tmp, ptr)
    join(first, ptr, last)
    return result
response ['', 'Verification failed', 'Errors:', 'Conditional statement might fail. There might be insufficient permission to access lseg(ptr, last). (tmp.py@41.11)', 'Verification took 2.18 seconds.']
response [''

### Create a jsonl file for training

In [ ]:
KEY = "tree"
dataset = data.Data("config.json", f"nagini_examples/{KEY}_master_finetune_full")
examples = dataset.get_list_of_examples(KEY)
print(len(examples))

def get_finetuning_prompt(example):
    unverified_program = dataset.get_example(KEY, example, "unverified")
    verif_error = dataset.get_cached_result(KEY, example)
    verified_program = dataset.get_example(KEY, example, "verified")
    prompt = f"""{dataset.get_system_prompt()}
### Unverified program:
{unverified_program}

### Verification error:
{verif_error}

### Verified program:
{verified_program}
"""
    return prompt


# create a jsonl file
hold_out = {
    "list": ["insert", "remove_last", "drop_iter", "count", "split", "merge_sort"],
    "tree": ["insert", "height", "min_depth"],
    "lseg": ["remove_last", "count_iter", "index_of", "reverse", "count", "split", "merge", "merge_sort", "insert_sorted_iter", "merge_iter"]
}
start_index = {
    "list": 18,
    "tree": 11,
    "lseg": 23
}
training_examples = {
    "list": [
        "prepend",
        "append",
        "remove_first",
        "join_lists",
        "contains",
        "remove",
        "index_of",
        "drop",
        "reverse",
        "insert_sorted",
        "insertion_sort",
        "merge",
    ],
    "tree": [
        "val_head",
        "count",
        "sum",
        "contains",
        "inorder",
        "min",
        "mirror",
        "subtree",
    ],
    "lseg": [
        "join",
        "prepend",
        "remove_first",
        "contains",
        "contains_iter",
        "insert",
        "insert_iter",
        "append",
        "append_iter",
        "insert_sorted",
        "insertion_sort",
        "insertion_sort_iter",
        "split_iter",
    ],
}
import jsonlines
with jsonlines.open('train_master.jsonl', mode='a') as writer:
    for i, example in enumerate(examples[start_index[KEY]:]):
        if not any(example.startswith(ex) for ex in training_examples[KEY]): 
            continue
        verif_error = dataset.get_cached_result(KEY, example)
        if verif_error == "Verification successful": 
            continue
        writer.write({
            "key": f"{KEY}_{example}",
            "prompt": get_finetuning_prompt(example)
        })